<a href="https://colab.research.google.com/github/ericburdett/cs601r-dl/blob/master/DL_Lab2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 2: Hyper Zoo

### Deliverable:
For this lab, you will submit an ipython notebook via learningsuite. Your notebook will contain two parts, as described below.

### Grading standards:
Your notebook will be graded on the following:

* 35% Part 1: Clearly displayed 10 bars (one for baseline, one for each tweak 
independently)
* 5% Part 1: Small writeup of conclusions from independent tweaks
* 25% Part 2: Clear explanation of your tweaking strategy
* 25% Part 2: Actually run your tweaking strategy and show the results
* 10% Tidy and legible figures, including labeled axes where appropriate
* 10% Extra credit - Error bars on your figure in Part 1.

### Description:
The goal of this lab is to learn how to explore the combinatorial space of possible hyperparameter settings.

Many deep learning papers present some sort of tweak on standard deep learning, and empirically illustrate that it improves performance (ideally across a wide variety of architectures and datasets). It quickly becomes hard to know: which, if any, of these tweaks are truly important - and how do they work when combined?

For this lab, you will explore various tweaks to the basic classifier you coded in lab 1. There are two parts to the lab.

### Part 1
You must clearly show the individual effect of each tweak compared to the baseline. For this part, you should present a simple bar chart (or possibly two or more, depending on your layout), clearly labeled with the baseline performance, and then the performance of each tweak relative to baseline. You may plot absolute or relative performances; whichever is clearer.

You must include a few sentences describing what you can conclude from evaluating all of these tweaks.

Note: I am not requiring error bars for this lab, because they are computationally intensive. I have made them extra credit – although if we were doing this for real, they would be absolutely required!

### Part 2
You must think about how to find the best combination of tweaks. There is no right answer to this part; I want you to think carefully about how to search the space of possible combinations, and come up with a reasonable method for settling on a final combination of tweaks. I have tried to provide enough tweaks that it should be impossible to brute-force try all possible combinations (although that is certainly a valid strategy!).

For this part, you must include in your notebook a simple writeup describing your strategy (just a paragraph or two), and then show the final performance of whatever combination you hit upon.

Note that you will not be graded on absolute performance of any run; what is important is thinking clearly through which tweaks make a difference.

### The Tweaks
Your baseline classifier must be a “vanilla” classifier, with none of the features listed below. We will systematically add them in.

You must test the following:

* Activation functions: relu (baseline), leakyrelu, selu, elu, hardshrink
* Batchnorm: off (baseline), on (use one batchnorm per residual block)
* Label smoothing: off (baseline), on
* Learning rate: constant (baseline), CLR
* Regularization: off (baseline), dropout
* Initialization: xavier/he (baseline), orthogonal

So, for part one, your bar chart should have 10 different bars.

Some of these tweaks require additional parameters. You should either leave them at their default values, or think of some reasonable way to set them.

Note: pytorch does not (AFAIK) natively implement label smoothing. In the interests of focusing on hyperparameter searching, you may verbatim copy any internet code you like to help implement label smoothing.

### Hints
Activation functions and dropout can all be found in torch.nn

Initialization functions can be found in torch.nn.init

This lab should be pretty straightforward, with the right script – you should be able to iterate over tweaks and run your classifier in a tidy loop. Ideally, you'll code it up, let it run, and come back in a few hours to find the results!

If you find yourself cutting-and-pasting, you might want to rethink your strategy.

## Imports

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import matplotlib.pyplot as plt
from torchvision import transforms, utils, datasets
from tqdm import tqdm
from torch.nn.parameter import Parameter
import pdb
import torchvision
import os
import gzip
import tarfile
from PIL import Image, ImageOps
import gc
import pdb
from IPython.core.ultratb import AutoFormattedTB
__ITB__ = AutoFormattedTB(mode = 'Verbose',color_scheme='LightBg', tb_offset = 1)

assert torch.cuda.is_available(), "Request a GPU from Runtime > Change Runtime"

## Model Implementation - 20 Layer Resnet

In [0]:
# For testing purposes - To get a baseline for performance
class PytorchResnet(nn.Module):
  def __init__(self, num_classes):
    super(PytorchResnet, self).__init__()

    self.model = torchvision.models.resnet152(pretrained=True)

    fc = nn.Linear(in_features=2048, out_features=num_classes)
    fc.requires_grad=True
    self.model.fc = fc

  def forward(self, x):
    return self.model(x)

In [0]:
class ResidualBlock(nn.Module):
  def __init__(self, in_channels, out_channels, batch_norm=True, activation=nn.ReLU, init_orthogonal=False, dropout=False):
    super(ResidualBlock, self).__init__()

    self.batch_norm = batch_norm
    self.dropout = dropout
    self.activation_function = activation
    self.activation = activation()

    self.conv1 = nn.Conv2d(in_channels, out_channels, 3, padding=1)
    torch.nn.init.orthogonal_(self.conv1.weight)
    self.bn1 = nn.BatchNorm2d(out_channels)
    self.drop = nn.Dropout(p=0.2)
    self.conv2 = nn.Conv2d(out_channels, out_channels, 3, padding=1)
    self.bn2 = nn.BatchNorm2d(out_channels)

    if init_orthogonal:
      torch.init.orthogonol_(self.conv1.weight)
      torch.init.orthogonol_(self.conv2.weight)

    if in_channels != out_channels:
      self.needs_shortcut = True
      self.conv_short = nn.Conv2d(in_channels, out_channels, 1, bias=False)
      self.bn_short = nn.BatchNorm2d(out_channels)

      if init_orthogonal:
        torch.init.orthogonal_(self.conv_short.weight)

      if batch_norm:
        self.shortcut = nn.Sequential(
          self.conv_short,
          self.bn_short
        )
      else:
        self.shortcut = self.conv_short

    else:
      self.needs_shortcut = False

  def forward(self, x):
    identity = x if not self.needs_shortcut else self.shortcut(x)
    out = self.conv1(x)

    if self.batch_norm:
      out = self.bn1(out)

    out = self.activation(out)

    if self.dropout:
      out = self.drop(out)

    out = self.conv2(out)

    if self.batch_norm:
      out = self.bn2(out)

    out += identity
    final_activation = self.activation_function()

    return final_activation(out)


In [0]:
class Resnet(nn.Module):
  def __init__(self, dataset, num_classes, batch_norm=False, activation=nn.ReLU, init_orthogonal=False, dropout=False):
    super(Resnet, self).__init__()

    x, y = dataset[0]
    in_channels, height, width = x.size()
    cstart = 64

    layers = []
    layers.append(ResidualBlock(in_channels, cstart, batch_norm=batch_norm, activation=activation, init_orthogonal=init_orthogonal, dropout=dropout))
    for i in range(19):
      layers.append(ResidualBlock(cstart, cstart, batch_norm=batch_norm, activation=activation, init_orthogonal=init_orthogonal, dropout=dropout))
  
    self.net = nn.Sequential(*layers)
    self.fc1 = nn.Linear(cstart * height * width, num_classes)

  def forward(self, x):
    # Residual Blocks
    out = self.net(x)

    # Flatten and narrow down to the number of classes
    out = torch.flatten(out, 1)
    out = self.fc1(out)

    return out

## Training Loop

In [0]:
class LabelSmoothLoss(nn.Module):
    def __init__(self, smoothing=0.0):
        super(LabelSmoothLoss, self).__init__()
        self.smoothing = smoothing
    
    def forward(self, input, target):
        log_prob = F.log_softmax(input, dim=-1)
        weight = input.new_ones(input.size()) * \
            self.smoothing / (input.size(-1) - 1.)
        weight.scatter_(-1, target.unsqueeze(-1), (1. - self.smoothing))
        loss = (-weight * log_prob).sum(dim=-1).mean()
        return loss

In [0]:
def train(activation=nn.ReLU, batch_norm=False, label_smoothing=False, clr=False, dropout=False, init_orthogonal=False):
  NUM_EPOCHS = 1
  BATCH_SIZE = 300

  train_dataset = torchvision.datasets.CIFAR10('/content/cifar-train', train=True, download=True, transform=transforms.Compose([transforms.ToTensor()]))
  val_dataset = torchvision.datasets.CIFAR10('/content/cifar-val', train=False, download=True, transform=transforms.Compose([transforms.ToTensor()]))

  model = Resnet(train_dataset, 10, batch_norm=batch_norm, activation=activation, init_orthogonal=init_orthogonal, dropout=dropout) #PytorchResnet(dataset.num_classes())
  model = model.cuda()

  if label_smoothing:
    objective = LabelSmoothLoss(smoothing=0.00)
  else:
    objective = LabelSmoothLoss(smoothing=0.05)


  optimizer = optim.SGD(model.parameters(), lr=1e-4)

  if clr:
    scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer, base_lr=1e-4, max_lr=1e-2)

  train_loader = DataLoader(train_dataset,
                            batch_size=BATCH_SIZE,
                            num_workers=4,
                            shuffle=True)
  val_loader = DataLoader(val_dataset,
                          batch_size=BATCH_SIZE,
                          num_workers=4,
                          shuffle=True)
  
  accuracy = 0
  
  predictions = np.array([])
  actual = np.array([])
  
  for epoch in range(NUM_EPOCHS):
    loop = tqdm(total=len(train_loader), position=0, leave=True)

    for batch, (x, y_truth) in enumerate(train_loader):
      gc.collect()
      x, y_truth = x.cuda(async=True), y_truth.cuda(async=True)

      optimizer.zero_grad()
      y_hat = model(x)

      loss = objective(y_hat, y_truth.long())

      loss.backward()

      loop.set_description('Train - epoch:{}, loss:{:.4f}'.format(epoch, loss.item()))
      loop.update(1)

      optimizer.step()
      if clr:
        scheduler.step()

    if epoch == (NUM_EPOCHS - 1):
      with torch.no_grad():

        val_single_acc = []

        model.eval()

        for _, (val_x, val_y_truth) in enumerate(val_loader):
          gc.collect()
          val_x, val_y_truth = val_x.cuda(), val_y_truth.cuda()

          val_y_hat = model(val_x)
          val_acc = torch.eq(val_y_hat.argmax(1), val_y_truth.long()).float().mean()

          val_single_acc.append(val_acc.item())

        accuracy = np.mean(val_single_acc)
        print('\nValidation - epoch:{}, acc:{:.4f}'.format(epoch, accuracy))

    loop.close()

  return accuracy

## Part I Loop

In [59]:
try:
  baseline = train(activation=nn.ReLU, batch_norm=False, label_smoothing=False, clr=False, dropout=False, init_orthogonal=False)
except:
  __ITB__()

Files already downloaded and verified
Files already downloaded and verified


Train - epoch:0, loss:2.3025: 100%|██████████| 167/167 [03:43<00:00,  1.20s/it]


Validation - epoch:0, acc:0.1004


In [0]:
# Obtain accuracies for all tweaks:
baseline = train(activation=nn.ReLU, batch_norm=False, label_smoothing=False, clr=False, dropout=False, init_orthogonal=False)
leakyrelu = train(activation=nn.LeakyReLU, batch_norm=False, label_smoothing=False, clr=False, dropout=False, init_orthogonal=False)
selu = train(activation=nn.SELU, batch_norm=False, label_smoothing=False, clr=False, dropout=False, init_orthogonal=False)
elu = train(activation=nn.ELU, batch_norm=False, label_smoothing=False, clr=False, dropout=False, init_orthogonal=False)
hardshrink = train(activation=nn.Hardshrink, batch_norm=False, label_smoothing=False, clr=False, dropout=False, init_orthogonal=False)
batchnorm = train(activation=nn.ReLU, batch_norm=True, label_smoothing=False, clr=False, dropout=False, init_orthogonal=False)
labelsmoothing = train(activation=nn.ReLU, batch_norm=False, label_smoothing=True, clr=False, dropout=False, init_orthogonal=False)
clr = train(activation=nn.ReLU, batch_norm=False, label_smoothing=False, clr=True, dropout=False, init_orthogonal=False)
dropout = train(activation=nn.ReLU, batch_norm=False, label_smoothing=False, clr=False, dropout=True, init_orthogonal=False)
orthogonal = train(activation=nn.ReLU, batch_norm=False, label_smoothing=False, clr=False, dropout=False, init_orthogonal=True)

In [0]:
import matplotlib.pyplot as plt
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
labels = ['Baseline', 'LeakyReLU', 'SELU', 'ELU', 'HardShrink', 'BatchNorm', 'LabelSmoothing', 'CLR', 'Dropout', 'Orthogonal']
accuracies = [baseline, leakyrelu, selu, elu, hardshrink, batchnorm, labelsmoothing, clr, dropout, orthogonal]
ax.bar(labels, accuracies)
plt.show()